In [3]:
import pandas
from datetime import datetime, timedelta
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from matplotlib import pyplot as plt
from keras import regularizers

%matplotlib inline

In [4]:
xls_file, hub_name, weather_station, load_station = "test_data.xlsx", "MICHIGAN.HUB", "Detroit", "north"

In [ ]:
df = parse_hub_data(hub_name=hub_name, xls_file=xls_file, 
                    weather_station=weather_station, load_station=load_station)

In [ ]:
def extract_day_of_week(df):
    
    return [d.isoweekday() for d in df.index]

In [ ]:
def extract_lagged_prices(df, delta):
    index = np.unique(df.index - delta)
    return df.loc[index]["rtlmp"].values
    

In [ ]:
def extract_demand_gradient(df):
    
    data = df["mtlf"]
    output = data - np.roll(data.values, 1)
    
    output[0] = np.nan
    
    return output
    

In [ ]:
def extract_features(df):
    
    output = pandas.DataFrame(index=df.index)
    
    output["rtlmp"] = df["rtlmp"]
    output["forecast_temperature"] = df["temperature"]
    output["forecast_load"] = df["mtlf"]
    output["hour"] = df["hour"]
    
    output["day_of_week"] = extract_day_of_week(df)
    
    output["change_in_demand_prediction"] = extract_demand_gradient(df)
    
    
    output["t1"] = extract_lagged_prices(df, np.timedelta64(1, "D"))
    output["t2"] = extract_lagged_prices(df, np.timedelta64(26, "h"))
    output["t3"] = extract_lagged_prices(df, np.timedelta64(27, "h"))
    output["t4"] = extract_lagged_prices(df, np.timedelta64(7, "D"))
    output["t5"] = extract_lagged_prices(df, np.timedelta64(7*24 -1, "h"))
    output["t6"] = extract_lagged_prices(df, np.timedelta64(7*24+1, "h"))
    output["t7"] = extract_lagged_prices(df, np.timedelta64(14, "D"))
    output["t8"] = extract_lagged_prices(df, np.timedelta64(21, "D"))
    output["t9"] = extract_lagged_prices(df, np.timedelta64(28, "D"))  
    
    return output.dropna()

In [ ]:
def split_into_training_data(df, training_cutoff_dates):
    
    X = df[[i for i in filter(lambda x: "rtlmp" not in x, df.columns)]]
    y = df["rtlmp"]
    
    from sklearn import preprocessing
    norm = preprocessing.MinMaxScaler(feature_range=[-1,1]).fit(X.values).transform
    normy = preprocessing.MinMaxScaler(feature_range=[-1,1]).fit(y.values)
    norm2=normy.transform
    
    
    training_cutoff = (df.index < training_cutoff_dates[1]) & (df.index > training_cutoff_dates[0])
    return norm(X.loc[training_cutoff].values), norm(X.loc[~training_cutoff].values), norm2(y.loc[training_cutoff].values), norm2(y.loc[~training_cutoff].values), normy.inverse_transform

In [ ]:
d = parse_hub_data(xls_file, hub_name, weather_station, load_station)

In [ ]:
features = extract_features(d)

In [ ]:
Xtrain, Xtest, ytrain, ytest, norm = split_into_training_data(features, [np.datetime64("2016-01-01"),np.datetime64("2016-08-05")])

In [ ]:
def build_model(): 
    model = Sequential()

    model.add(Dense(units=10, input_dim=Xtrain.shape[1]))
    
    model.add(Activation('tanh'))
    model.add(Dense(units=5, kernel_regularizer=regularizers.l1(0.0001)))
    model.add(Activation('tanh'))
    model.add(Dense(units=1, activation="linear"))

    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model

In [ ]:
estimator.fit(Xtrain, ytrain)

In [ ]:
yhat = norm(estimator.predict(Xtest))

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
reg = RidgeCV()
reg.fit(Xtrain, ytrain)
yhat2 = norm(reg.predict(Xtest))
 

In [ ]:
for i in range(40):
    plt.figure()
    plt.plot(yhat[i*7*24:(i+1)*7*24], 'b-')
    plt.plot(yhat2[i*7*24:(i+1)*7*24], 'g-')
    plt.plot(norm(ytest[i*7*24:(i+1)*7*24]), 'k-')

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
encoding_dim = 10

input_vector = Input(shape=(Xtest.shape[1],))

# add a Dense layer with a L1 activity regularizer
encoded = Dense(encoding_dim, activation='relu',
                activity_regularizer=regularizers.l1(10e-5))(input_vector)
decoded = Dense(784, activation='sigmoid')(encoded)

autoencoder = Model(input_vector, decoded)